# Import Libraries

In [3]:
import os
import sys
from tqdm import tqdm
import random
import osmnx as ox
import networkx as nx
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
import pickle

import shutil
from matplotlib import pyplot as plt

import warnings; warnings.simplefilter('ignore')

pd.options.display.max_columns = None
pd.options.display.max_rows = 30
np.set_printoptions(threshold=sys.maxsize)

## Extract road network dataset from OpenStreetMap (OSM)

In [ ]:
# Define coordinate reference system: WGS84 - World Geodetic System 1984 used in GPS
crs_lonlat = {'init': 'epsg:' + str(4326)}
# Directory where the boundaries for networks (in shapefile format) are stored
shapefile_bnd_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Global road network resilience/01_data/4229_bnd_convex'

# Extract OSM network using the boundary shapefiles
counter = 0
for i, file in enumerate(os.listdir(shapefile_bnd_dir)):
    if file[-4:] == '.shp':
        bnd = gpd.read_file(shapefile_bnd_dir+file).to_crs(crs_lonlat)
        net_id = bnd['Id'][0]
        try:                 
            G = ox.graph_from_polygon(bnd.geometry[0], network_type='drive_service')
            counter += 1

            with open('/05_WB/CityResilience/1_processed/raw_osmgraphs_drive_service_conv/g_raw_drive_conv_' + str(net_id) + '.pk', 'wb') as handle:
                pickle.dump(G, handle, protocol=2)
        except:
            print(file)

**Summarize network/graph properties:**
- Number of nodes
- Number of edges
- Tertiary road lengths
- Residential road lengths

In [ ]:
# Initiate summary dataframe
df = pd.DataFrame(columns=['net_id', 'nodes', 'edges','tot_length','tertiary_length','residential_length'], index=range(4194))
counter = 0

# Calculate the lengths for tertiary as well as residential road types
for file in os.listdir(network_dir):
    G = pickle.load(open(network_dir + file, 'rb'))
    tertiary_length = []
    residential_length = []
    net_id = file[6:][:-3]
    # Populate dataframe with network id, number of nodes, number of edges, total road length
    df.at[counter, 'net_id'] = net_id
    df.at[counter, 'nodes'] = G.number_of_nodes()
    df.at[counter, 'edges'] = G.number_of_edges()
    df.at[counter, 'tot_length'] = G.size(weight="length")
    # Examine the "highway" attribute of the OSM networks/graphs for road lengths
    for i,j,data in G.edges.data():
        if data['highway'] == 'tertiary':
            tertiary_length.append(data['length'])
        elif data['highway'] == 'tertiary_link':
            tertiary_length.append(data['length'])
        elif data['highway'] == 'residential':
            residential_length.append(data['length'])
    # Populate dataframe with tertiary road length and residential road length
    df.at[counter, 'tertiary_length'] = sum(tertiary_length)
    df.at[counter, 'residential_length'] = sum(residential_length)
    counter += 1
df.to_csv('/05_WB/CityResilience/1_processed/graphs_drive_service_tertiary_road_lengths_4194.csv')

### Examine different road types ("highway") in the OSM road network dataset

In [ ]:
# Function to remove list nestings
def reemovNestings(l): 
    for i in l: 
        if type(i) == list: 
            reemovNestings(i)
        else: 
            output.append(i)

# Initiate dataframe to store road types
df = pd.DataFrame(columns=['net_id', 'net_types'], index=range(4190))
counter = 0
net_type_lst = [] # Initiate list of network types

# For each network, summarize road type
for file in os.listdir(network_dir):
    G = pickle.load(open(network_dir + file, 'rb'))
    net_id = file[6:][:-3]
    df.at[counter, 'net_id'] = net_id
    highway_class = nx.get_edge_attributes(G,'highway')
    output = [] 
    reemovNestings(highway_class.values())
    df.at[counter, 'net_types'] = list(set(output))
    net_type_lst.append(list(set(output)))
    counter += 1
    
output = []
reemovNestings(net_type_lst)

road_type_count_df = pd.DataFrame(columns=['highway_class', 'count'], index=range(171))
counter = 0

for item in list(set(output)):
    road_type_count_df.at[counter, 'highway_class'] = item
    road_type_count_df.at[counter, 'count'] = output.count(item)
    counter += 1

road_type_count_df.to_csv('/05_WB/CityResilience/1_processed/Drive_service_conv_highway_class_count.csv')

## Remove residential roads

In [ ]:
# Initialize directory to store processed networks/graphs
network_dir = 'L:/yiyi/graphs_cov_no_res2/'

# Remove residential roads
for file in tqdm(os.listdir(network_dir)):
    net_id = file[15:][:-3]
    G = pickle.load(open(network_dir + file, 'rb'))
    G2 = G.copy()
    for i,j,data in G.edges.data():
        if data['highway'] == 'residential':
            G2.remove_edge(i, j)
        elif  'residential' in data['highway']:
            G2.remove_edge(i, j)
    # Remove isolated nodes as a result of the edge removal
    G2.remove_nodes_from(list(nx.isolates(G2)))
    with open('L:yiyi/graphs_cov_no_res3/graph_cov_no_r_' + str(net_id) + '.pk', 'wb') as handle:
                pickle.dump(G2, handle, protocol=2)

## Compare original convex hull area with graph convex hull area

In [ ]:
graph_convex_hull_shp_dir = 'L:/yiyi/4190_graph_cov/'
graph_conv_sqkm_df = pd.DataFrame(columns=['net_id', 'graph_conv_sqkm'], index=range(4188))
counter = 0

for file in tqdm(os.listdir(graph_convex_hull_shp_dir)):
    if file[-3:] == 'shp':
        net_id = int(file[2:][:-14])
        data = gpd.read_file(graph_convex_hull_shp_dir+file)
        data_meters = data.to_crs({'init': 'epsg:6933'})
        sqkm = data_meters['geometry'].area/ 10**6
        
        graph_conv_sqkm_df.at[counter, 'net_id'] = net_id
        graph_conv_sqkm_df.at[counter, 'graph_conv_sqkm'] = sqkm
        
        counter += 1
        
graph_conv_sqkm_df['graph_conv_sqkm_simplified'] = graph_conv_sqkm_df.apply(lambda row: float(row['graph_conv_sqkm']), axis=1)
graph_conv_sqkm_df[['net_id', 'graph_conv_sqkm_simplified']].to_csv('L:/yiyi/graph_4188_conv_sqkm.csv')

original_convex_hulls = gpd.read_file('H:/05_WB/CityResilience/1_processed/final_bnd/final_bnd/urcls_4190_conv_prop.shp')
original_convex_hulls_meters = original_convex_hulls.to_crs({'init': 'epsg:6933'})

In [ ]:
ori_graph_conv_merge = pd.merge(original_convex_hulls_meters[['net_id', 'python_sqkm']], graph_conv_sqkm_df[['net_id', 'graph_conv_sqkm_simplified']],
                               left_on = 'net_id', right_on = 'net_id', how = 'inner')
ori_graph_conv_merge['missing_percentage'] = ori_graph_conv_merge.apply(lambda row:
                                                                        (row['python_sqkm'] - row['graph_conv_sqkm_simplified'])*100/row['python_sqkm'],
                                                                        axis = 1)
num_graph_df = pd.DataFrame(columns=['missing_percentage', 'count_of_4188'])
for i in range(100):
    missing_percentage = i+1
    count = ori_graph_conv_merge[ori_graph_conv_merge['missing_percentage']<missing_percentage].shape[0]
    num_graph_df.at[i, 'missing_percentage'] = missing_percentage
    num_graph_df.at[i, 'count_of_4188'] = count

num_graph_df.to_csv('L:/yiyi/missing_percentage_count_of_4188.csv')

# KNEE POINT / ELBOW POINT
y = list(num_graph_df['count_of_4188'].values)
x = range(1, len(y)+1)

kn = KneeLocator(x, y, curve='concave', direction='increasing')
print(kn.knee)

## Summarize the number of nodes, edges and length of roads for 4190 networks
- nodes
- edges
- tot_length
- primary road length
- secondary road length
- tertiary road length
- motorway length

In [ ]:
df = pd.DataFrame(columns=['net_id', 'nodes', 'edges','tot_length',
                           'primary_length',
                           'secondary_length',
                           'tertiary_length', 
                           'motorway_length', 
                           'unclassified_length',
                           'service_length',
                           'trunk_length', 
                           'living_street_length',
                           'road_length',
                           'residential_length' ], index=range(4190))
counter = 0

for file in os.listdir(network_dir):
    G = pickle.load(open(network_dir + file, 'rb'))
    
    primary_length = []
    secondary_length = []
    tertiary_length = []
    motorway_length = []
    unclassified_length = []
    service_length = []
    trunk_length = []
    living_street_length = []
    road_length = []
    residential_length = []
    
    net_id = file[15:][:-3]
    df.at[counter, 'net_id'] = net_id
    df.at[counter, 'nodes'] = G.number_of_nodes()
    df.at[counter, 'edges'] = G.number_of_edges()
    df.at[counter, 'tot_length'] = G.size(weight="length")
    for i,j,data in G.edges.data():
        if data['highway'] == 'primary' or data['highway'] == 'primary_link':
            primary_length.append(data['length'])
            
        elif data['highway'] == 'secondary' or data['highway'] == 'secondary_link' :
            secondary_length.append(data['length'])
            
        elif data['highway'] == 'tertiary' or data['highway'] == 'tertiary_link':
            tertiary_length.append(data['length'])
        
        elif data['highway'] == 'motorway' or data['highway'] == 'motorway_link':
            motorway_length.append(data['length'])
            
        elif data['highway'] == 'unclassified':
            unclassified_length.append(data['length'])
        
        elif data['highway'] == 'service':
            service_length.append(data['length'])
        
        elif data['highway'] == 'trunk' or data['highway'] == 'trunk_link':
            trunk_length.append(data['length'])
            
        elif data['highway'] == 'living_street':
            living_street_length.append(data['length'])
        
        elif data['highway'] == 'road':
            road_length.append(data['length'])
        
        elif data['highway'] == 'residential':
            residential_length.append(data['length'])
            
    df.at[counter, 'primary_length'] = sum(primary_length) 
    df.at[counter, 'secondary_length'] = sum(secondary_length)
    df.at[counter, 'tertiary_length'] = sum(tertiary_length)
    df.at[counter, 'motorway_length'] = sum(motorway_length)
    df.at[counter, 'unclassified_length'] = sum(unclassified_length)
    df.at[counter, 'service_length'] = sum(service_length)
    df.at[counter, 'trunk_length'] = sum(trunk_length)
    df.at[counter, 'living_street_length'] = sum(living_street_length)
    df.at[counter, 'road_length'] = sum(road_length)
    df.at[counter, 'residential_length'] = sum(residential_length)
    counter += 1
    
df.to_csv('L:/yiyi/drive_service_no_res_4190_graph_properties.csv')